In [1]:
# from Model.BERT_BILSTM_CRF import BERTBILSTMCRF
from Model.BILSTM_Attetion_CRF import BILSTMAttentionCRF
from Model.BILSTM_CRF import BILSTMCRF
from Model.IDCNN_CRF import IDCNNCRF
from Model.IDCNN5_CRF import IDCNNCRF2

from sklearn.metrics import f1_score, recall_score
import numpy as np
import pandas as pd

from Public.utils import *
from keras.callbacks import EarlyStopping
from DataProcess.process_data import DataProcess

max_len = 100


def train_sample(train_model='BERTBILSTMCRF',
                 # ['BERTBILSTMCRF', 'BILSTMAttentionCRF', 'BILSTMCRF',
                 # 'IDCNNCRF', 'IDCNNCRF2']
                 epochs=15,
                 log = None,
                 ):

    # bert需要不同的数据参数 获取训练和测试数据
    if train_model == 'BERTBILSTMCRF':
        dp = DataProcess(data_type='msra', max_len=max_len, model='bert')
    else:
        dp = DataProcess(data_type='msra', max_len=max_len)
    train_data, train_label, test_data, test_label = dp.get_data(one_hot=True)

    log.info("----------------------------数据信息 START--------------------------")
    log.info(f"当前使用数据集 MSRA")
    # log.info(f"train_data:{train_data.shape}")
    log.info(f"train_label:{train_label.shape}")
    # log.info(f"test_data:{test_data.shape}")
    log.info(f"test_label:{test_label.shape}")
    log.info("----------------------------数据信息 END--------------------------")

    if train_model == 'BERTBILSTMCRF':
#         model_class = BERTBILSTMCRF(dp.vocab_size, dp.tag_size, max_len=max_len)
        pass
    elif train_model == 'BILSTMAttentionCRF':
        model_class = BILSTMAttentionCRF(dp.vocab_size, dp.tag_size)
    elif train_model == 'BILSTMCRF':
        model_class = BILSTMCRF(dp.vocab_size, dp.tag_size)
    elif train_model == 'IDCNNCRF':
        model_class = IDCNNCRF(dp.vocab_size, dp.tag_size, max_len=max_len)
    else:
        model_class = IDCNNCRF2(dp.vocab_size, dp.tag_size, max_len=max_len)

    model = model_class.creat_model()

    callback = TrainHistory(log=log, model_name=train_model)  # 自定义回调 记录训练数据
    early_stopping = EarlyStopping(monitor='val_crf_viterbi_accuracy', patience=2, mode='max')  # 提前结束
    model.fit(train_data, train_label, batch_size=32, epochs=epochs,
              validation_data=[test_data, test_label],
              callbacks=[callback, early_stopping])

    # 计算 f1 和 recall值

    pre = model.predict(test_data)
    pre = np.array(pre)
    test_label = np.array(test_label)
    pre = np.argmax(pre, axis=2)
    test_label = np.argmax(test_label, axis=2)
    pre = pre.reshape(pre.shape[0] * pre.shape[1], )
    test_label = test_label.reshape(test_label.shape[0] * test_label.shape[1], )

    f1score = f1_score(pre, test_label, average='macro')
    recall = recall_score(pre, test_label, average='macro')

    log.info("================================================")
    log.info(f"--------------:f1: {f1score} --------------")
    log.info(f"--------------:recall: {recall} --------------")
    log.info("================================================")

    # 把 f1 和 recall 添加到最后一个记录数据里面
    info_list = callback.info
    if info_list and len(info_list)>0:
        last_info = info_list[-1]
        last_info['f1'] = f1score
        last_info['recall'] = recall

    return info_list


if __name__ == '__main__':

    # 需要测试的模型
    train_modes = ['IDCNNCRF', 'IDCNNCRF2', 'BILSTMAttentionCRF', 'BILSTMCRF']

    # 定义文件路径（以便记录数据）
    log_path = os.path.join(path_log_dir, 'train_log.log')
    df_path = os.path.join(path_log_dir, 'df.csv')
    log = create_log(log_path)

    # 训练同时记录数据写入的df文件中
    columns = ['model_name','epoch', 'loss', 'acc', 'val_loss', 'val_acc', 'f1', 'recall']
    df = pd.DataFrame(columns=columns)
    for model in train_modes:
        info_list = train_sample(train_model=model, epochs=1, log=log)
        for info in info_list:
            df = df.append([info])
        df.to_csv(df_path)

Using TensorFlow backend.
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 

MSRA数据预处理已经完成




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          2704256   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 64)           24640     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 128)          24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 128)          49280     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 128)          0         
_________

/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '



Train on 37092 samples, validate on 9273 samples
Epoch 1/1
37092/37092 [==============================] - 107s 3ms/step - loss: 0.0948 - crf_viterbi_accuracy: 0.9723 - val_loss: 0.0406 - val_crf_viterbi_accuracy: 0.9858
MSRA数据预处理已经完成
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 128)          2704256   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 100, 256)          65792     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 100, 256)          196864    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 100, 512)          524800    
_______________________________________

/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Train on 37092 samples, validate on 9273 samples
Epoch 1/1
37092/37092 [==============================] - 270s 7ms/step - loss: 0.0744 - crf_viterbi_accuracy: 0.9764 - val_loss: 0.0347 - val_crf_viterbi_accuracy: 0.9868
MSRA数据预处理已经完成
WQ.shape (?, ?, 300)
K.permute_dimensions(WK, [0, 2, 1]).shape (?, 300, ?)
QK.shape (?, ?, ?)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 128)         2704256   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
attention_self_1 (AttentionS (None, None, 300)         230400    
_________________________________________________________________
dropout_4 (D

/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Train on 37092 samples, validate on 9273 samples
Epoch 1/1
37092/37092 [==============================] - 309s 8ms/step - loss: 0.1368 - crf_viterbi_accuracy: 0.9587 - val_loss: 0.0656 - val_crf_viterbi_accuracy: 0.9792
MSRA数据预处理已经完成
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 128)         2704256   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
dense_5 (Dense)              (None, None, 7)           1799      
________________________________________

/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


Train on 37092 samples, validate on 9273 samples
Epoch 1/1
37092/37092 [==============================] - 262s 7ms/step - loss: 0.1234 - crf_viterbi_accuracy: 0.9624 - val_loss: 0.0451 - val_crf_viterbi_accuracy: 0.9822
